In [599]:
import configparser
import requests
import datetime
from requests import Session
from psycopg2 import connect
from psycopg2.extras import execute_values
from time import sleep
CONFIG = configparser.ConfigParser()
CONFIG.read(r'C:\Users\nchan6\Documents\db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [214]:
def get_url(mapserver, id):
    "This prints the base_url for the request"
    url = "https://insideto-gis.toronto.ca/arcgis/rest/services/{}/MapServer/{}/query".format(mapserver, id)
    return(url)

In [608]:
def to_time(input):
    time = datetime.datetime.fromtimestamp(input/1000).strftime('%Y-%m-%d %H:%M:%S')
    return time

In [603]:
def check_status(max_number, record_max, output_table, insert_column):
    "This gets the request status"
    try:
        query = {"where": "1=1", 
                "outFields": "*",
                "outSR": '4326',         
                "returnGeometry": "true",
                "returnTrueCurves": "false",
                "returnIdsOnly": "false",
                "returnCountOnly": "false",
                "returnZ": "false",
                "returnM": "false",
                "returnDistinctValues": "false",
                "resultOffset": "{}".format(max_number),
                "resultRecordCount": "{}".format(record_max),
                "orderByFields": "OBJECTID", 
                "returnExtentsOnly": "false",
                "f":"json"}  

        print(query)
        print(max_number, record_max)
        r = requests.get(base_url, params = query, verify = False)
    except requests.exceptions.SSLError:
        sleep(5)
        error_status = 'somethings up'
        rule = 'dont add'
        status = (error_status, rule)
    else:
        return_json = r.json()
        features = return_json['features']
        fields = return_json['fields']
        trials = [[field['name'], field['type']] for field in fields]
        rows = []
        for feature in features:
            geometry = 'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['paths'][0]) +')'
            row = [feature['attributes'][trial[0]] if trial[1] != 'esriFieldTypeDate' else datetime.datetime.fromtimestamp((feature['attributes'][trial[0]])/1000).strftime('%Y-%m-%d %H:%M:%S') for trial in trials]
            row.append(geometry)
            rows.append(row)          

        sql='INSERT INTO {} {} VALUES %s'.format(output_table, insert_column)

        with con:
            with con.cursor() as cur:
                execute_values(cur, sql, rows)   
                
        if return_json.get('exceededTransferLimit', False) == True:
            rule = 'add'
        else:
            rule = 'dont add'  
            
        error_status = 'success'
        status = (error_status, rule)    
    return status

In [609]:
def get_layer(mapserver, id, output_table):
    
    """
    This function calls to the GCCview rest API and inserts the outputs to the output table in the postgres database.

    Parameters
    ----------
    mapserver : string
        The name of the mapserver that host the desire layer

    id : int
        The id of desire layer

    output_table: string
        Name of table that returned row will be inserted into
        
    """  
    base_url = get_url({}, {}).format(mapserver, id)
    query = {"where":"1=1",
             "outFields": "*",
             "outSR": '4326',         
             "returnGeometry": "true",
             "returnTrueCurves": "false",
             "returnIdsOnly": "false",
             "returnCountOnly": "false",
             "returnZ": "false",
             "returnM": "false",
             "orderByFields": "OBJECTID", 
             "returnDistinctValues": "false",
             "returnExtentsOnly": "false",
             "f":"json"}
    
    # First get request to get maxnumber and append first request
    r = requests.get(base_url, params = query, verify = False)
    return_json = r.json()
    features = return_json['features']
    record_max=(len(features))
    max_number = record_max
    rows = []
    
    # Create a table in postgres based on returned fields
    fields = return_json['fields']
    new_column = '('
    insert_column= '('
    for field in fields:
        column_name = (field['name'].lower()).replace('.', '_')
        if field['type'] == 'esriFieldTypeInteger' or field['type'] == 'esriFieldTypeInteger' or field['type'] =='esriFieldTypeOID' or field['type'] == 'esriFieldTypeSmallInteger' or field['type'] =='esriFieldGlobalID':
                column_type = 'integer'
        elif field['type'] == 'esriFieldTypeString':
                column_type = 'text'
        elif field['type'] == 'esriFieldTypeDouble':
                column_type = 'numeric'
        elif field['type'] == 'esriFieldTypeDate':
                column_type = 'timestamp without time zone'
                               
        new_column = new_column + column_name +' '+column_type+', '
        insert_column = insert_column + column_name +','
        
    new_column = new_column +'geom geometry)' 
    insert_column = insert_column + 'geom)'
    
    sql= '''create table {} {}'''.format(output_table, new_column)
    
    with con:
        with con.cursor() as cur:
            cur.execute(sql)  
    
    # We first append the first set of features    
    trials = [[field['name'], field['type']] for field in fields]
    rows = []
         
    # Set up geometry for point, line, and polygon  
    for feature in features:
        if return_json['geometryType'] == 'esriGeometryPolyline' or return_json['geometryType'] == 'esriGeometryLine':
            geometry = 'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['paths'][0]) +')'
        
        elif return_json['geometryType'] =='esriGeometryPoint':
            geometry = 'SRID=4326;Point('+(str(feature['geometry']['x']))+' '+ (str(feature['geometry']['y']))+')'                       
        
        elif return_json['geometryType'] =='esriGeometryPolygon' or return_json['geometryType'] =='esriGeometryMultiPolygon':
            geometry = 'SRID=4326;MultiPolygon((('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['rings'][0]) +')))'    
        
        row = [feature['attributes'][trial[0]] if trial[1] != 'esriFieldTypeDate' or feature['attributes'][trial[0]] == None else to_time(feature['attributes'][trial[0]]) for trial in trials]
        row.append(geometry)
        rows.append(row)          
    
    sql='INSERT INTO {} {} VALUES %s'.format(output_table, insert_column)
        
    with con:
        with con.cursor() as cur:
            execute_values(cur, sql, rows)        
    
    if return_json.get('exceededTransferLimit', False) == True:
        error_status = 'successful'
        rule = 'add'
    else:
        error_status = 'successful'
        rule = 'dont add'
    status = (error_status, rule)
    
    print(status)
    print(max_number, record_max)
    if status[1] == 'add':
        while status[0] =='successful' or status[0] == 'somethings up' or status[1] == 'add':

            status = check_status(max_number, record_max, output_table, insert_column)

            if status[0] == 'successful' or status[1] == 'add':
                max_number = max_number + record_max
            elif status[0] == 'somethings up' or status[1] == 'dont add':
                continue 
                
            print(max_number)
    else:
        print('all rows inserted')

In [602]:
base_url = get_url('cot_geospatial2', 5 )
query = {"where":"1=1",
             "outFields": "*",
             "outSR": '4326',         
             "returnGeometry": "true",
             "returnTrueCurves": "false",
             "returnIdsOnly": "false",
             "returnCountOnly": "false",
             "returnZ": "false",
             "returnM": "false",
             "orderByFields": "OBJECTID", 
             "returnDistinctValues": "false",
             "returnExtentsOnly": "false",
             "f":"json"}
    

r = requests.get(base_url, params = query, verify = False)       
return_json = r.json()
features = return_json['features']
fields = return_json['fields']
keys = [field['name'] for field in fields]
trials = [[field['name'], field['type']] for field in fields]


rows = []
fee=[]
fees =[]
fields = return_json['fields']
for feature in features:
    if return_json['geometryType'] == 'esriGeometryPolyline' or return_json['geometryType'] == 'esriGeometryLine':
        geometry = 'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['paths'][0]) +')'
        
    elif return_json['geometryType'] =='esriGeometryPoint':
        geometry = 'SRID=4326;Point('+(str(feature['geometry']['x']))+' '+ (str(feature['geometry']['y']))+')'                       
        
    elif return_json['geometryType'] =='esriGeometryPolygon' or return_json['geometryType'] =='esriGeometryMultiPolygon':
        geometry = 'SRID=4326;MultiPolygon((('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['rings'][0]) +')))'    
         

    row = [feature['attributes'][trial[0]] if trial[1] != 'esriFieldTypeDate' else datetime.datetime.fromtimestamp((feature['attributes'][trial[0]])/1000).strftime('%Y-%m-%d %H:%M:%S') for trial in trials]
    row.append(geometry)
    rows.append(row)    

print(rows)
    


[['YONGE ST', '197 YONGE ST', None, None, 'Toronto and East York', 'Major Arterial', '2017-11-14 16:36:00', 'Non-Expressways', 43.653532, -79.379448, '2015-09-30 20:01:00', '2019-12-31 18:59:00', 'Continuous', 'Northbound curb lane closed due to construction.', 'Planned', 'Y', 'N', 55492, 55492, 'SRID=4326;Point(-79.37944799856419 43.653531999254504)'], ['YORKVILLE AVE', 'YONGE STREET', None, None, 'Toronto and East York', 'Collector', '2017-11-14 16:26:00', 'Non-Expressways', 43.671977, -79.387453, '2016-04-30 20:01:00', '2019-06-30 19:59:00', 'Continuous', 'Southbound curb lane closed due to construction.', 'Planned', 'Y', 'N', 55672, 55672, 'SRID=4326;Point(-79.38745300484473 43.67197700423177)'], ['LOMBARD ST', None, 'RICHMOND ST E', 'VICTORIA ST', 'Toronto and East York', 'Collector', '2017-11-14 16:23:00', 'Non-Expressways', 43.651396999999996, -79.377375, '2016-09-14 20:01:00', '2019-12-31 18:59:00', 'Continuous', 'Northbound portion of street closed due to construction.', 'Plan

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [611]:
get_layer('cot_geospatial2', 5, 'gcc_table1')

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


('successful', 'dont add')
623 623
all rows inserted
